***

*Course:* [Math 535](https://people.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS)  
*Chapter:* 7-Probabilistic models   
*Author:* [Sebastien Roch](https://people.math.wisc.edu/~roch/), Department of Mathematics, University of Wisconsin-Madison  
*Updated:* April 12, 2024   
*Copyright:* &copy; 2024 Sebastien Roch

***

In [ ]:
# You will need the files:
#     * mmids.py
# from https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils
#
# IF RUNNING ON GOOGLE COLAB (RECOMMENDED):
# "Upload to session storage" from the Files tab on the left
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
# PYTHON 3
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import mmids
seed = 535
rng = np.random.default_rng(seed)

## Motivating example: tracking location

Suppose we let loose a cyborg corgi in a large park. We would like to know where it is at all time. For this purpose, it has an implanted location device that sends a signal to a tracking app.   

Here is an example of the data we might have. The red dots are recorded locations at regular time intervals. The dotted line helps keep track of the time order of the recordings. (We will explain later how this dataset is generated.)

In [ ]:
ss = 4
os = 2
F = np.array([[1., 0., 1., 0.],[0., 1., 0., 1.],[0., 0., 1., 0.],[0., 0., 0., 1.]]) 
H = np.array([[1., 0., 0., 0.],[0., 1, 0., 0.]])
Q = 0.1 * np.diag(np.ones(ss))
R = 10 * np.diag(np.ones(os))
init_mu = np.array([0., 0., 1., 1.])
init_Sig = 1 * np.diag(np.ones(ss))
T = 50
x, y = mmids.lgSamplePath(ss, os, F, H, Q, R, init_mu, init_Sig, T)
plt.plot(y[0,:], y[1,:], marker='o', c='r', linestyle='dotted')
plt.xlim((np.min(y[0,:])-5, np.max(y[0,:])+5)) 
plt.ylim((np.min(y[1,:])-5, np.max(y[1,:])+5))
plt.show()

By convention, we start at $(0,0)$. Notice how squiggly the trajectory is. One issue might be that the times  at which the location is recorded are too far between. But, in fact, there is another issue: the tracking device is *inaccurate*. 

To get a better estimate of the true trajectory, it is natural to try to model the noise in the measurement as well as the dynamics itself. Probabilistic models are perfectly suited for this. 

In this chapter, we will encounter of variety of such models and show how to take advantage of them to estimate unknown states (or parameters). In particular, conditional independence will play a key role.

We will come back to location tracking later in the chapter.

$\newcommand{\P}{\mathbb{P}}$ $\newcommand{\E}{\mathbb{E}}$ $\newcommand{\S}{\mathcal{S}}$ $\newcommand{\X}{\mathcal{X}}$ $\newcommand{\var}{\mathrm{Var}}$ $\newcommand{\btheta}{\boldsymbol{\theta}}$ $\newcommand{\bbeta}{\boldsymbol{\beta}}$ $\newcommand{\bphi}{\boldsymbol{\phi}}$ $\newcommand{\bpi}{\boldsymbol{\pi}}$ $\newcommand{\bmu}{\boldsymbol{\mu}}$ $\newcommand{\bSigma}{\boldsymbol{\Sigma}}$ $\newcommand{\balpha}{\boldsymbol{\alpha}}$ $\newcommand{\indep}{\perp\!\!\!\perp}$ $\newcommand{\bLambda}{\boldsymbol{\Lambda}}$

## Background: introduction to parametric families and maximum likelihood estimation

### Exponential family

**EXAMPLE:** **(Multivariate Gaussian)** A multivariate Gaussian vector $\mathbf{X} = (X_1,\ldots,X_d)$ on $\mathbb{R}^d$ with mean $\bmu \in \mathbb{R}^d$ and positive definite covariance matrix $\bSigma \in \mathbb{R}^{d \times d}$ has probability density function 

$$
f_{\bmu, \bSigma}(\mathbf{x})
= \frac{1}{(2\pi)^{d/2} \,|\bSigma|^{1/2}}
\exp\left(-\frac{1}{2}(\mathbf{x} - \bmu)^T \bSigma^{-1} (\mathbf{x} - \bmu)\right).
$$

We use the notation $\mathbf{X} \sim N_d(\bmu, \bSigma)$.

It can be shown that indeed the mean is

$$
\E[\mathbf{X}]
= \bmu
$$

and the covariance matrix is

$$
\E[(\mathbf{X} - \bmu)(\mathbf{X} - \bmu)^T]
= \E[\mathbf{X} \mathbf{X}^T] - \bmu \bmu^T
= \bSigma.
$$

See, e.g., [[Bis](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf), Secion 2.3].

The following code, which plots the density in the bivariate case, was adapted from [gauss_plot_2d.ipynb](https://github.com/probml/pyprobml/blob/master/notebooks/book1/03/gauss_plot_2d.ipynb) by ChatGPT.

**LEARNING BY CHATTING:** Ask your favorite AI chatbot to explain the code! Try different covariance matrices. $\ddagger$

In [ ]:
from scipy.stats import multivariate_normal

def gaussian_pdf(X, Y, mean, cov):
    xy = np.stack([X.flatten(), Y.flatten()], axis=-1)
    return multivariate_normal.pdf(
        xy, mean=mean, cov=cov).reshape(X.shape)

In [ ]:
def make_surface_plot(X, Y, Z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(
        X, Y, Z, cmap=plt.cm.viridis, antialiased=False)
    plt.show()

We plot the density for mean $(0,0)$ with two different covariance matrices:

$$
\bSigma_1 = \begin{bmatrix} 1.0 & 0 \\ 0 & 1.0 \end{bmatrix}
\quad \text{and} \quad 
\bSigma_2 = \begin{bmatrix} \sigma_1^2 & \rho \sigma_1 \sigma_2 \\ \rho \sigma_1 \sigma_2 & \sigma_2^2 \end{bmatrix}
$$

where $\sigma_1 = 1.5$, $\sigma_2 = 0.5$ and $\rho = -0.75$.

In [ ]:
start_point = 5
stop_point = 5
num_samples = 100
points = np.linspace(-start_point, stop_point, num_samples)
X, Y = np.meshgrid(points, points)

mean = np.array([0., 0.])
cov = np.array([[1., 0.], [0., 1.]])
make_surface_plot(X, Y, gaussian_pdf(X, Y, mean, cov))

In [ ]:
mean = np.array([0., 0.])
cov = np.array([[1.5 ** 2., -0.75 * 1.5 * 0.5], 
                 [-0.75 * 1.5 * 0.5, 0.5 ** 2.]])
make_surface_plot(X, Y, gaussian_pdf(X, Y, mean, cov))

Rewriting the density as

$$
f_{\bmu, \bSigma}(\mathbf{x})
= \frac{e^{-(1/2) \bmu^T \bSigma^{-1} \bmu}}{(2\pi)^{d/2} \,|\bSigma|^{1/2}}
\exp\left(- \mathbf{x}^T \bSigma^{-1}\bmu 
- \frac{1}{2} \mathrm{tr}\left(\mathbf{x} \mathbf{x}^T \bSigma^{-1}\right)\right)
$$

where we used the symmetric nature of $\bSigma^{-1}$ in the first term of the exponential and the previous trace identity in the second term. The expression in parentheses is linear in the entries of $\mathbf{x}$ and $\mathbf{x} \mathbf{x}^T$, which can then be taken as  sufficient statistics (formally, using [vectorization](https://en.wikipedia.org/wiki/Vectorization_%28mathematics%29)). Indeed note that

$$
\mathbf{x}^T \bSigma^{-1}\bmu 
= \sum_{i=1}^d x_i (\bSigma^{-1}\bmu)_i
$$

and

$$
\mathrm{tr}\left(\mathbf{x} \mathbf{x}^T \bSigma^{-1}\right)
= \sum_{i = 1}^d \left(\sum_{j=1}^d (\mathbf{x} \mathbf{x}^T)_{i,j} (\bSigma^{-1})_{j,i}\right)
= \sum_{i = 1}^d \sum_{j=1}^d x_i x_j (\bSigma^{-1})_{j,i}.
$$

So we can take 

$$
\bphi(\mathbf{x})
= (x_1,\ldots,x_d, x_1 x_1, \ldots, x_d x_1, x_1 x_2, \ldots, x_d x_2, \ldots, x_1 x_d, \ldots, x_d x_d)
$$

\begin{align*}
\btheta
&= \bigg(-(\bSigma^{-1}\bmu)_1,\ldots,-(\bSigma^{-1}\bmu)_d,\\
&\qquad - \frac{1}{2}(\bSigma^{-1})_{1,1}, \ldots, - \frac{1}{2}(\bSigma^{-1})_{1,d},\\
&\qquad - \frac{1}{2}(\bSigma^{-1})_{2,1}, \ldots, - \frac{1}{2}(\bSigma^{-1})_{2,d},\\ 
&\qquad \ldots, - \frac{1}{2}(\bSigma^{-1})_{d,1}, \ldots,- \frac{1}{2}(\bSigma^{-1})_{d,d}\bigg)
\end{align*}

and $h (\mathbf{x}) \equiv 1$. Expressing $Z(\btheta)$ explicitly is  not straightforward. But note that $\btheta$ includes all entries of $\bSigma^{-1}$, from which $\bSigma$ can be computed (e.g., from [Cramer's rule](https://en.wikipedia.org/wiki/Cramer%27s_rule#Finding_inverse_matrix)), and in turn from which $\bmu$ can be extracted out of the entries of $\bSigma^{-1}\bmu$ in $\btheta$. So the normalizing factor $\frac{(2\pi)^{d/2} \,|\bSigma|^{1/2}}{e^{-(1/2) \bmu^T \bSigma^{-1} \bmu}}$ can in principle be expressed in terms of $\btheta$.

This shows that the multivariate normal is an exponential family. 

The matrix $\bLambda = \bSigma^{-1}$ is also known as the precision matrix.

Alternatively, let $\mathbf{Z}$ be a standard Normal $d$-vector,
let $\bmu \in \mathbb{R}^d$ and let $\bSigma \in \mathbb{R}^{d \times d}$ be positive definite. Then the transformed random variable $\mathbf{X} = \bmu + \bSigma \mathbf{Z}$ is a multivariate Gaussian with mean $\bmu$ and covariance matrix $\bSigma$. This can be proved using the [change of variables formula](https://en.wikipedia.org/wiki/Probability_density_function#Function_of_random_variables_and_change_of_variables_in_the_probability_density_function) (try it!). $\lhd$

**NUMERICAL CORNER:** In Numpy, as we have seen before, the module [numpy.random](https://numpy.org/doc/stable/reference/random/index.html) provides a way to sample from a variety of standard distributions. We first initialize the [pseudorandom number generator](https://en.wikipedia.org/wiki/Pseudorandom_number_generator) with a [random seed](https://en.wikipedia.org/wiki/Random_seed). Recall that it allows the results to be reproducible: using the same seed produces the same results again.

In [ ]:
seed = 535
rng = np.random.default_rng(535)

Here's are lists of available [probability distributions](https://numpy.org/doc/stable/reference/random/generator.html#distributions).

In [ ]:
p = 0.1 # probability of success
N = 5 # number of samples
rng.binomial(1, p, size=N) # Bernoulli is special case of binomial with 1 trial

Here are a few other examples.

In [ ]:
p = [0.1, 0.2, 0.7]
n = 100
rng.multinomial(n, p, size=N)

In [ ]:
mu = np.array([0.1, -0.3])
sig = np.array([[2., 0.],[0., 3.]])
rng.multivariate_normal(mu, sig, size=N)

$\unlhd$

### Parameter estimation

### Parameter estimation for exponential families

### Generalized linear models

$\newcommand{\P}{\mathbb{P}}$ $\newcommand{\E}{\mathbb{E}}$ $\newcommand{\S}{\mathcal{S}}$ $\newcommand{\var}{\mathrm{Var}}$ $\newcommand{\btheta}{\boldsymbol{\theta}}$ $\newcommand{\bphi}{\boldsymbol{\phi}}$ $\newcommand{\bpi}{\boldsymbol{\pi}}$ $\newcommand{\bmu}{\boldsymbol{\mu}}$ $\newcommand{\blambda}{\boldsymbol{\lambda}}$ $\newcommand{\bSigma}{\boldsymbol{\Sigma}}$ $\newcommand{\balpha}{\boldsymbol{\alpha}}$ $\newcommand{\indep}{\perp\!\!\!\perp}$ $\newcommand{\bp}{\mathbf{p}}$ $\newcommand{\bx}{\mathbf{x}}$ $\newcommand{\bX}{\mathbf{X}}$ $\newcommand{\by}{\mathbf{y}}$ $\newcommand{\bY}{\mathbf{Y}}$ $\newcommand{\bz}{\mathbf{z}}$ $\newcommand{\bZ}{\mathbf{Z}}$ $\newcommand{\bw}{\mathbf{w}}$ $\newcommand{\bW}{\mathbf{W}}$ $\newcommand{\pa}{\mathrm{pa}}$

## Modeling more complex dependencies

### Trick 1: Imposing conditional independence relations

### Trick 2: Marginalizing out an unobserved random variable

**EXAMPLE:** **(Gaussian mixture model)** For $i=1,\ldots,K$, let $\bmu_i$ and $\bSigma_i$ be the mean and covariance matrix of a multivariate Gaussian. Let $\bpi \in \Delta_K$. A Gaussian Mixture Model (GMM) is obtained as follows: take $Y \sim \mathrm{Cat}(\bpi)$ and

$$
\bX|\{Y=i\} \sim N_d(\bmu_i, \bSigma_i).
$$

Its probability density function (PDF) takes the form

$$
f_\bX(\bx)
= \sum_{i=1}^K \pi_i \frac{1}{(2\pi)^{d/2} \,|\bSigma_i|^{1/2}}
\exp\left(-\frac{1}{2}(\mathbf{x} - \bmu_i)^T \bSigma_i^{-1} (\bx - \bmu_i)\right).
$$

This is illustrated below.

We plot the density for means $\bmu_1 = (-2,-2)$ and $\bmu_2 = (2,2)$ and covariance matrices

$$
\bSigma_1 = \begin{bmatrix} 1.0 & 0 \\ 0 & 1.0 \end{bmatrix}
\quad \text{and} \quad 
\bSigma_2 = \begin{bmatrix} \sigma_1^2 & \rho \sigma_1 \sigma_2 \\ \rho \sigma_1 \sigma_2 & \sigma_2^2 \end{bmatrix}
$$

where $\sigma_1 = 1.5$, $\sigma_2 = 0.5$ and $\rho = -0.75$. The mixing weights are $\pi_1 = 0.25$ and $\pi_2 = 0.75$.

In [ ]:
from scipy.stats import multivariate_normal

def gmm2_pdf(X, Y, mean1, cov1, pi1, mean2, cov2, pi2):
    xy = np.stack([X.flatten(), Y.flatten()], axis=-1)
    Z1 = multivariate_normal.pdf(
        xy, mean=mean1, cov=cov1).reshape(X.shape) 
    Z2 = multivariate_normal.pdf(
        xy, mean=mean2, cov=cov2).reshape(X.shape) 
    return pi1 * Z1 + pi2 * Z2

In [ ]:
start_point = 6
stop_point = 6
num_samples = 100
points = np.linspace(-start_point, stop_point, num_samples)
X, Y = np.meshgrid(points, points)

mean1 = np.array([-2., -2.])
cov1 = np.array([[1., 0.], [0., 1.]])
pi1 = 0.5
mean2 = np.array([2., 2.])
cov2 = np.array([[1.5 ** 2., -0.75 * 1.5 * 0.5], 
                 [-0.75 * 1.5 * 0.5, 0.5 ** 2.]])
pi2 = 0.5
Z = gmm2_pdf(X, Y, mean1, cov1, pi1, mean2, cov2, pi2)
mmids.make_surface_plot(X, Y, Z)

We could also consider continuous mixing distributions. Several standard approaches, e.g. probabilistic PCA, factor analysis and independent components analysis, use such mixtures. See [[Bis](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf), Chapter 12]. $\lhd$

**NUMERICAL CORNER:** In Numpy, as we have seen before, the module [numpy.random](https://numpy.org/doc/stable/reference/random/index.html) also provides a way to sample from mixture models by using [numpy.random.Generator.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html). This is better seen on an example.

In [ ]:
seed = 535 # setting the seed
rng = np.random.default_rng(seed) 

In [ ]:
# constructs a mixture of three normal distributions,
# with prior probabilities [0.2, 0.5, 0.3]
mus = np.array([-2.0, 0.0, 3.0])
sigmas = np.array([1.2, 1.0, 2.5])
coeffs = np.array([0.2, 0.5, 0.3])

In [ ]:
N = 5 # number of samples
which = rng.choice(3,N,p=coeffs)
rng.normal(mus[which],sigmas[which])

More generally, we consider mixtures of multivariate Gaussians. We chage the notation slightly to track Python's indexing. For $i=0,1$, we have a mean $\bmu_i \in \mathbb{R}^d$ and a positive definite covariance matrix $\bSigma_i \in \mathbb{R}^{d \times d}$. We also have mixture weights $\phi_0, \phi_1 \in (0,1)$ such that $\phi_0 + \phi_1 = 1$. Suppose we want to generate a total of $n$ samples.

For each sample $j=1,\ldots, n$, independently from everything else:

1. We first pick a component $i \in \{0,1\}$ at random according to the mixture weights, that is, $i=0$ is chosen with probability $\phi_0$ and $i=1$ is chosen with probability $\phi_1$.

2. We generate a sample $\bX_j = (X_{j,1},\ldots,X_{j,d})$ according to a multivariate Gaussian with mean $\bmu_i$ and covariance $\bSigma_i$. 

This is straightforward to implement by using again [`numpy.random.Generator.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html) to choose the component of each sample and [`numpy.random.Generator.multivariate_normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.multivariate_normal.html) to generate multivariate Gaussians. For convenience, we will stack the means and covariances into one array with a new dimension. So, for instance, the covariance matrices will now be in a 3d-array, that is, an array with $3$ indices. The first index corresponds to the component (here $0$ or $1$).

In [ ]:
d = 2
sigma0 = np.outer(np.array([2., 2.]), np.array([2., 2.])) 
sigma0 += np.outer(np.array([-0.5, 0.5]), np.array([-0.5, 0.5]))
sigma1 = 2 * np.identity(d)
sigma = np.stack((sigma0,sigma1))

In [ ]:
print(sigma[0,:,:])

In [ ]:
print(sigma[1,:,:])

Here is a visualization borrowed from the [TensorFlow tutorial](https://www.tensorflow.org/guide/tensor#basics).

**Figure:** Three matrices ([Source](https://www.tensorflow.org/guide/tensor#basics))

![Three matrices](https://www.tensorflow.org/static/guide/images/tensor/3-axis_numpy.png)

$\bowtie$

**Figure:** Three matrices stacked into a 3d-array ([Source](https://www.tensorflow.org/guide/tensor#basics))


![Three matrices stacked into a tensor](https://www.tensorflow.org/static/guide/images/tensor/3-axis_front.png)

$\bowtie$

The code is the following. It returns an `d` by `n` array `X`, where each row is a sample from a 2-component Gaussian mixture. 

In [ ]:
def gmm2(d, n, phi0, phi1, mu0, sigma0, mu1, sigma1):
    
    # merge components into tensors
    phi = np.stack((phi0, phi1))
    mu = np.stack((mu0, mu1))
    sigma = np.stack((sigma0,sigma1))
    
    # initialization
    X = np.zeros((n,d))
    
    # choose components of each data point, then generate samples
    component = rng.choice(2, size=n, p=phi)
    for i in range(n):
        X[i,:] = rng.multivariate_normal(
            mu[component[i],:],
            sigma[component[i],:,:])
    
    return X

Let us try it with following parameters.

In [ ]:
d, n, w = 2, 1000, 5.
phi0 = 0.9
phi1 = 0.1
mu0 = np.concatenate(([w], np.zeros(d-1)))
mu1 = np.concatenate(([-w], np.zeros(d-1)))
sigma0 = np.outer(np.array([2., 2.]), np.array([2., 2.])) 
sigma0 += np.outer(np.array([-0.5, 0.5]), np.array([-0.5, 0.5]))
sigma1 = 2 *np.identity(d)

In [ ]:
X = gmm2(d, n, phi0, phi1, mu0, sigma0, mu1, sigma1)

In [ ]:
plt.scatter(X[:,0], X[:,1], marker='x')
plt.axis('equal')
plt.show()

$\unlhd$

### Example 1: Naive Bayes

**NUMERICAL CORNER:** We implement the Naive Bayes model. We use [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) to avoid overfitting issues.

We use a simple example from [Towards Data Science](https://towardsdatascience.com/all-about-naive-bayes-8e13cef044cf):

> **Example:** let’s say we have data on 1000 pieces of fruit. The fruit being a Banana, Orange or some other fruit and imagine we know 3 features of each fruit, whether it’s long or not, sweet or not and yellow or not, as displayed in the table below.

![Table](./figs/1*B_uXqox7nHfwTa1HH4Fc_A.png)

> [...] Which should provide enough evidence to predict the class of another fruit as it’s introduced.

We encode the data into a table, where the rows are the classes and the columns are the features. The entries are the corresponding $N_{k,m}$s. In addition we provide the vector $N_k$, which is the last column above, and the value $N$, which is the sum of the entries of $N_k$. 

In [ ]:
N_km = np.array([[400., 350., 450.],
                 [0., 150., 300.],
                 [100., 150., 50.]])

In [ ]:
def nb_fit_table(N_km, alpha=1., beta=1.):
    
    K, M = N_km.shape
    N_k = np.sum(N_km,axis=-1)
    N = np.sum(N_k)
    
    # MLE for pi_k's
    pi_k = (N_k+alpha) / (N+K*alpha)
    
    # MLE for p_km's
    p_km = (N_km+beta) / (N_k[:,None]+2*beta)

    return pi_k, p_km

We run it on our simple dataset.

In [ ]:
pi_k, p_km = nb_fit_table(N_km)

In [ ]:
print(pi_k)

In [ ]:
print(p_km)

Continuing on with our previous example:

> So let’s say we’re given the features of a piece of fruit and we need to predict the class. If we’re told that the additional fruit is Long, Sweet and Yellow, we can classify it using the [prediction] formula and subbing in the values for each outcome, whether it’s a Banana, an Orange or Other Fruit. The one with the highest probability (score) being the winner.

The next function computes the negative logarithm of $\pi_k \prod_{m=1}^M p_{k,m}^{x_m} (1-p_{k,m})^{1-x_m}$, that is, the score of $k$, and outputs a $k$ achieving the minimum score.

In [ ]:
def nb_predict(pi_k, p_km, x, label_set):
   
    K = len(pi_k)
    
    # Computing the score for each k
    score_k = np.zeros(K)
    for k in range(K):
       
        score_k[k] -= np.log(pi_k[k])
        score_k[k] -= np.sum(x * np.log(p_km[k,:]) 
                               + (1 - x)*np.log(1 - p_km[k,:]))
    
    # Computing the minimum
    argmin = np.argmin(score_k, axis=0)
    minscr = np.max(score_k, axis=0)

    return label_set[argmin]

We run it on our dataset with the additional fruit from the quote above.

In [ ]:
label_set = ['Banana', 'Orange', 'Other']
x = np.array([1., 1., 1.])

In [ ]:
nb_predict(pi_k, p_km, x, label_set)

$\unlhd$

### Example 2: Mixtures of multivariate Bernoullis and the EM algorithm

**NUMERICAL CORNER:** We implement the EM algorithm for mixtures of multivariate Bernoullis. For this purpose, we adapt our previous Naive Bayes routines. We also allow for the possibility of using Laplace smoothing.

In [ ]:
def responsibility(pi_k, p_km, x):
   
    K = len(pi_k)
        
    # Computing the score for each k
    score_k = np.zeros(K)
    for k in range(K):
       
        score_k[k] -= np.log(pi_k[k])
        score_k[k] -= np.sum(x*np.log(p_km[k,:]) 
                             + (1 - x)*np.log(1 - p_km[k,:]))
    
    # Computing responsibilities for each k
    r_k = np.exp(-score_k)/(np.sum(np.exp(-score_k)))
        
    return r_k

In [ ]:
def update_parameters(eta_km, eta_k, eta, alpha, beta):

    K = len(eta_k)
    
    # MLE for pi_k's
    pi_k = (eta_k+alpha) / (eta+K*alpha)
    
    # MLE for p_km's
    p_km = (eta_km+beta) / (eta_k[:,None]+2*beta)

    return pi_k, p_km

We implement the E and M Step next.

In [ ]:
def em_bern(X, K, pi_0, p_0, maxiters = 10, alpha=0., beta=0.):
    
    n, M = X.shape
    pi_k = pi_0
    p_km = p_0
    
    for _ in range(maxiters):
    
        # E Step
        r_ki = np.zeros((K,n))
        for i in range(n):
            r_ki[:,i] = responsibility(pi_k, p_km, X[i,:])
        
        # M Step     
        eta_km = np.zeros((K,M))
        eta_k = np.sum(r_ki, axis=-1)
        eta = np.sum(eta_k)
        for k in range(K):
            for m in range(M):
                eta_km[k,m] = np.sum(X[:,m] * r_ki[k,:]) 
        pi_k, p_km = update_parameters(
            eta_km, eta_k, eta, alpha, beta)
        
    return pi_k, p_km   

We test the algorithm on a very simple dataset.

In [ ]:
X = np.array([[1., 1., 1.],
              [1., 1., 1.],
              [1., 1., 1.],
              [1., 0., 1.],
              [0., 1., 1.],
              [0., 0., 0.],
              [0., 0., 0.],
              [0., 0., 1.]])
n, M = X.shape
K = 2

In [ ]:
seed = 535
rng = np.random.default_rng(seed)
pi_0 = np.ones(K)/K
p_0 = rng.random((K,M))

In [ ]:
pi_k, p_km = em_bern(
    X, K, pi_0, p_0, maxiters=100, alpha=0.01, beta=0.01)

In [ ]:
print(pi_k)

In [ ]:
print(p_km)

We compute the probability that the vector $(0, 0, 1)$ is in each cluster.

In [ ]:
x_test = np.array([0., 0., 1.])
responsibility(pi_k, p_km, x_test)

To give a more involved example, we return to the MNIST dataset. There are two common ways to write a $2$. Let's see if a mixture of multivariate Bernoullis can find them. We load the dataset and extract the images labelled $2$.

In [ ]:
# Download and load the MNIST dataset
mnist = datasets.MNIST(root='./data', 
                       train=True, 
                       download=True, 
                       transform=transforms.ToTensor())

# Convert the dataset to a PyTorch DataLoader
train_loader = torch.utils.data.DataLoader(mnist, 
                                           batch_size=len(mnist), 
                                           shuffle=False)

# Extract images and labels from the DataLoader
imgs, labels = next(iter(train_loader))
imgs = imgs.squeeze().numpy()
labels = labels.numpy()

In [ ]:
# Filter out images with label 2
mask = labels == 2
imgs2 = imgs[mask]
labels2 = labels[mask]

Next, we transform the images into vectors and convert into black and white by rounding.

In [ ]:
X = np.round(imgs2.reshape(len(imgs2), -1))

We can convert back as follows.

In [ ]:
plt.figure()
plt.imshow(X[0,:].reshape((28,28)))
plt.show()

In this example, the probabilities involved are very small and the responsibilities are close to $0$ or $1$. We use a variant, called hard EM, which replaces responsibilities with the one-hot encoding of the largest responsibility.

In [ ]:
def hard_responsibility(pi_k, p_km, x):
   
    K = len(pi_k)
        
    # Computing the score for each k
    score_k = np.zeros(K)
    for k in range(K):
       
        score_k[k] -= np.log(pi_k[k])
        score_k[k] -= np.sum(x*np.log(p_km[k,:]) 
                             + (1 - x)*np.log(1 - p_km[k,:]))
    
    # Computing responsibilities for each k
    argmin = np.argmin(score_k, axis=0)
    r_k = np.zeros(K)
    r_k[argmin] = 1

    return r_k

In [ ]:
def hard_em_bern(X, K, pi_0, p_0, maxiters = 10, alpha=0., beta=0.):
    
    n, M = X.shape
    pi_k = pi_0
    p_km = p_0
    
    for _ in range(maxiters):
    
        # E Step
        r_ki = np.zeros((K,n))
        for i in range(n):
            r_ki[:,i] = hard_responsibility(pi_k, p_km, X[i,:])
        
        # M Step     
        eta_km = np.zeros((K,M))
        eta_k = np.sum(r_ki, axis=-1)
        eta = np.sum(eta_k)
        for k in range(K):
            for m in range(M):
                eta_km[k,m] = np.sum(X[:,m] * r_ki[k,:]) 
        pi_k, p_km = update_parameters(
            eta_km, eta_k, eta, alpha, beta)
        
    return pi_k, p_km   

We run the algorithm with $2$ clusters. You may have to run it a few times to get a meaningful clustering.

In [ ]:
n, M = X.shape
K = 2
pi_0 = np.ones(K)/K
p_0 = rng.random((K,M))

In [ ]:
pi_k, p_km = hard_em_bern(
    X, K, pi_0, p_0, maxiters=10, alpha=1., beta=1.)

In [ ]:
print(pi_k)

We vizualize the two uncovered clusters by rendering their means as an image. Here is one of them.

In [ ]:
plt.figure()
plt.imshow(p_km[0,:].reshape((28,28)))
plt.show()

Here is the other one.

In [ ]:
plt.figure()
plt.imshow(p_km[1,:].reshape((28,28)))
plt.show()

$\unlhd$

$\newcommand{\P}{\mathbb{P}}$ $\newcommand{\E}{\mathbb{E}}$ $\newcommand{\S}{\mathcal{S}}$ $\newcommand{\var}{\mathrm{Var}}$ $\newcommand{\bmu}{\boldsymbol{\mu}}$ $\newcommand{\bSigma}{\boldsymbol{\Sigma}}$ $\newcommand{\btheta}{\boldsymbol{\theta}}$ $\newcommand{\bpi}{\boldsymbol{\pi}}$ $\newcommand{\indep}{\perp\!\!\!\perp}$ $\newcommand{\bp}{\mathbf{p}}$ $\newcommand{\bx}{\mathbf{x}}$ $\newcommand{\bX}{\mathbf{X}}$ $\newcommand{\by}{\mathbf{y}}$ $\newcommand{\bY}{\mathbf{Y}}$ $\newcommand{\bz}{\mathbf{z}}$ $\newcommand{\bZ}{\mathbf{Z}}$ $\newcommand{\bw}{\mathbf{w}}$ $\newcommand{\bW}{\mathbf{W}}$ $\newcommand{\bv}{\mathbf{v}}$ $\newcommand{\bV}{\mathbf{V}}$ $\newcommand{\bLambda}{\boldsymbol{\Lambda}}$ $\newcommand{\bu}{\mathbf{u}}$ $\newcommand{\bU}{\mathbf{U}}$

## Linear-Gaussian models and Kalman filtering

### Multivariate Gaussians: marginals and conditionals

### Kalman filter

### Location tracking

We apply Kalman filtering to location tracking. Returning to our cyborg corgi example, we imagine that we get noisy observations about its successive positions in a park. (Think of GPS measurements.) We seek to get a better estimate of its location using the method above. See for example this [blog post](https://towardsdatascience.com/optimal-estimation-algorithms-kalman-and-particle-filters-be62dcb5e83) on location tracking.

We model the true location as a linear-Gaussian system over the 2d position $(z_{1t}, z_{2t})_t$ and velocity $(\dot{z}_{1t}, \dot{z}_{2t})_t$ sampled at $\Delta$ intervals of time. Formally, the system is 

$$
\bX_t = (z_{1t}, z_{2t}, \dot{z}_{1t}, \dot{z}_{2t}),
\quad
F = \begin{pmatrix}
1 & 0 & \Delta & 0\\
0 & 1 & 0 & \Delta\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{pmatrix}.
$$

In words, the velocity is unchanged, up to Gaussian perturbation. The position changes proportionally to the velocity in the corresponding dimension.

The observations $(\tilde{z}_{1t}, \tilde{z}_{2t})_t$ are modeled as

$$
\bY_t = (\tilde{z}_{1t}, \tilde{z}_{2t}),
\quad 
H = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
\end{pmatrix}.
$$

In words, we only observe the positions, up to Gaussian noise.

**Implementing the Kalman filter** We implement the Kalman filter as described above with known covariance matrices. We take $\Delta = 1$ for simplicity. The code is adapted from [[Mur](https://github.com/probml)].

We will test Kalman filtering on a simulated path drawn from the linear-Gaussian model above. The following function creates such a path and its noisy observations.

In [ ]:
seed = 535
rng = np.random.default_rng(seed)

In [ ]:
def lgSamplePath(ss, os, F, H, Q, R, init_mu, init_Sig, T):
    x = np.zeros((ss,T)) 
    y = np.zeros((os,T))

    x[:,0] = rng.multivariate_normal(init_mu, init_Sig)
    for t in range(1,T):
        x[:,t] = rng.multivariate_normal(F @ x[:,t-1],Q) # noise on x_t
        y[:,t] = rng.multivariate_normal(H @ x[:,t],R) # noise on y_t
    
    return x, y

Here is an example. Here $\bSigma$ is denoted as $V$. In the plot, the blue crosses are the unobserved true path and the orange dots are the noisy observations.

In [ ]:
ss = 4 # state size
os = 2 # observation size
F = np.array([[1., 0., 1., 0.],
              [0., 1., 0., 1.],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.]]) 
H = np.array([[1., 0., 0., 0.],
              [0., 1, 0., 0.]])
Q = 0.1 * np.diag(np.ones(ss))
R = 10 * np.diag(np.ones(os))
init_mu = np.array([0., 0., 1., 1.])
init_Sig = 1 * np.diag(np.ones(ss))
T = 50
x, y = lgSamplePath(ss, os, F, H, Q, R, init_mu, init_Sig, T)

In [ ]:
plt.plot(y[0,:], y[1,:], marker='o', c='r', linestyle='dotted')
plt.xlim((np.min(y[0,:])-5, np.max(y[0,:])+5)) 
plt.ylim((np.min(y[1,:])-5, np.max(y[1,:])+5))
plt.show()

In [ ]:
plt.plot(x[0,:], x[1,:], 
         marker='x', c='g', linestyle='dashed', alpha=0.5)
plt.xlim((np.min(x[0,:])-5, np.max(x[0,:])+5)) 
plt.ylim((np.min(x[1,:])-5, np.max(x[1,:])+5))
plt.scatter(y[0,:], y[1,:], c='r')
plt.show()

The following function implements the Kalman filter. The full recursion is broken up into several steps. We use [`numpy.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) to compute the Kalman gain matrix.

In [ ]:
def kalmanUpdate(ss, F, H, Q, R, y_t, mu_prev, Sig_prev):
    mu_pred = F @ mu_prev
    Sig_pred = F @ Sig_prev @ F.T + Q
    e_t = y_t - H @ mu_pred # error at time t
    S = H @ Sig_pred @ H.T + R
    Sinv = LA.inv(S)
    K = Sig_pred @ H.T @ Sinv # Kalman gain matrix
    mu_new = mu_pred + K @ e_t
    Sig_new = (np.diag(np.ones(ss)) - K @ H) @ Sig_pred
    return mu_new, Sig_new

In [ ]:
def kalmanFilter(ss, os, y, F, H, Q, R, init_mu, init_Sig, T):
    mu = np.zeros((ss, T))
    Sig = np.zeros((ss, ss, T))
    mu[:,0] = init_mu
    Sig[:,:,0] = init_Sig

    for t in range(1,T):
        mu[:,t], Sig[:,:,t] = kalmanUpdate(ss, F, H, Q, R, 
                                           y[:,t], mu[:,t-1], 
                                           Sig[:,:,t-1])

    return mu, Sig

We apply this to the example above. The inferred unobserved states are in green.

In [ ]:
init_mu = np.array([0., 0., 1., 1.])
init_Sig = 1 * np.diag(np.ones(ss))
mu, Sig = kalmanFilter(ss, os, y, F, H, Q, R, init_mu, init_Sig, T)

In [ ]:
plt.plot(x[0,:], x[1,:], 
         marker='x', c='g', linestyle='dashed', alpha=0.5)
plt.xlim((np.min(x[0,:])-5, np.max(x[0,:])+5)) 
plt.ylim((np.min(x[1,:])-5, np.max(x[1,:])+5))
plt.scatter(y[0,:], y[1,:], c='r')
plt.plot(mu[0,:], mu[1,:], marker='s', linewidth=2)
plt.show()

To quantify the improvement in the inference compared to the observations, we compute the mean squared error in both cases.

In [ ]:
dobs = x[0:1,:] - y[0:1,:]
mse_obs = np.sqrt(np.sum(dobs**2))
print(mse_obs)

In [ ]:
dfilt = x[0:1,:] - mu[0:1,:]
mse_filt = np.sqrt(np.sum(dfilt**2))
print(mse_filt)

We indeed observe a reduction.

**Missing data** We can also allow for the possibility that some observations are missing. Imagine for instance losing GPS signal while going through a tunnel. The recursions above are still valid, with the only modification that the $\bY_t$ and $H$ terms are dropped at those times $t$ where there is no observation. In Numpy, we can use [`NaN`](https://numpy.org/doc/stable/reference/constants.html#numpy.nan). (Alternatively, one can use the [numpy.ma](https://numpy.org/doc/stable/reference/maskedarray.generic.html) module.) 

We use a same sample path as above, but mask observations at times $t=10,\ldots,20$.

In [ ]:
ss = 4
os = 2
F = np.array([[1., 0., 1., 0.],
              [0., 1., 0., 1.],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.]]) 
H = np.array([[1., 0., 0., 0.],
              [0., 1, 0., 0.]])
Q = 0.01 * np.diag(np.ones(ss))
R = 10 * np.diag(np.ones(os))
init_mu = np.array([0., 0., 1., 1.])
init_Sig = Q
T = 30
x, y = lgSamplePath(ss, os, F, H, Q, R, init_mu, init_Sig, T)

In [ ]:
for i in range(10,20):
    y[0,i] = np.nan
    y[1,i] = np.nan

Here is the sample we are aiming to infer.

In [ ]:
plt.plot(x[0,:], x[1,:], 
         marker='x', c='g', 
         linestyle='dashed', alpha=0.5)
plt.xlim((np.min(x[0,:])-5, np.max(x[0,:])+5)) 
plt.ylim((np.min(x[1,:])-5, np.max(x[1,:])+5))
plt.scatter(y[0,:], y[1,:], c='r')
plt.show()

We modify the recursion accordingly.

In [ ]:
def kalmanUpdate(ss, F, H, Q, R, y_t, mu_prev, Sig_prev):
    mu_pred = F @ mu_prev
    Sig_pred = F @ Sig_prev @ F.T + Q
    if np.isnan(y_t[0]) or np.isnan(y_t[1]):
        return mu_pred, Sig_pred
    else:
        e_t = y_t - H @ mu_pred # error at time t
        S = H @ Sig_pred @ H.T + R
        Sinv = LA.inv(S)
        K = Sig_pred @ H.T @ Sinv # Kalman gain matrix
        mu_new = mu_pred + K @ e_t
        Sig_new = (np.diag(np.ones(ss)) - K @ H) @ Sig_pred
        return mu_new, Sig_new

In [ ]:
init_mu = np.array([0., 0., 1., 1.])
init_Sig = 1 * np.diag(np.ones(ss))
mu, Sig = kalmanFilter(ss, os, y, F, H, Q, R, init_mu, init_Sig, T)

In [ ]:
plt.plot(x[0,:], x[1,:], 
         marker='x', c='g', 
         linestyle='dashed', alpha=0.5)
plt.xlim((np.min(x[0,:])-5, np.max(x[0,:])+5)) 
plt.ylim((np.min(x[1,:])-5, np.max(x[1,:])+5))
plt.scatter(y[0,:], y[1,:], c='r')
plt.plot(mu[0,:], mu[1,:], marker='s', linewidth=2)
plt.show()

$\newcommand{\P}{\mathbb{P}}$ $\newcommand{\E}{\mathbb{E}}$ $\newcommand{\S}{\mathcal{S}}$ $\newcommand{\var}{\mathrm{Var}}$ $\newcommand{\bmu}{\boldsymbol{\mu}}$ $\newcommand{\bSigma}{\boldsymbol{\Sigma}}$ $\newcommand{\btheta}{\boldsymbol{\theta}}$ $\newcommand{\bpi}{\boldsymbol{\pi}}$ $\newcommand{\indep}{\perp\!\!\!\perp}$ $\newcommand{\bp}{\mathbf{p}}$ $\newcommand{\bx}{\mathbf{x}}$ $\newcommand{\bX}{\mathbf{X}}$ $\newcommand{\by}{\mathbf{y}}$ $\newcommand{\bY}{\mathbf{Y}}$ $\newcommand{\bz}{\mathbf{z}}$ $\newcommand{\bZ}{\mathbf{Z}}$ $\newcommand{\bw}{\mathbf{w}}$ $\newcommand{\bW}{\mathbf{W}}$ $\newcommand{\bv}{\mathbf{v}}$ $\newcommand{\bV}{\mathbf{V}}$ $\newcommand{\Z}{\mathcal{Z}}$ $\newcommand{\bh}{\mathbf{h}}$ $\newcommand{\bb}{\mathbf{b}}$ $\newcommand{\bc}{\mathbf{c}}$ $\newcommand{\cE}{\mathcal{E}}$ $\newcommand{\cP}{\mathcal{P}}$

## Gibbs sampling with application to generating images

### Markov chain Monte Carlo (MCMC)

**Sampling from simple distributions** When $\bpi$ is a standard distribution or $\S$ is relatively small, this can be done efficiently by using a random number generator, as we have done previously.

**NUMERICAL CORNER:** Recall how this works. We first initialize the random number generator and use a `seed` for reproducibility.

In [ ]:
seed = 535
rng = np.random.default_rng(seed)

To generate, say $1000$, samples from a multivariate normal, say with mean $(0, 0)$ and covariance $\begin{pmatrix}5 & 0\\0 & 1\end{pmatrix}$, we use [`numpy.random.Generator.multivariate_normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.multivariate_normal.html#numpy.random.Generator.multivariate_normal) as follows.

In [ ]:
mean = np.array([0., 0.])
cov = np.array([[5., 0.], [0., 1.]])
x, y = rng.multivariate_normal(mean, cov, 1000).T

Plotting the result we get:

In [ ]:
plt.scatter(x, y, marker='x')
plt.axis('equal')
plt.show()

Computing the mean of each component we get:

In [ ]:
print(np.mean(x))

In [ ]:
print(np.mean(y))

This is somewhat close to the expected answer: $(0,0)$. 

Using a larger number of samples, say $10,000$, gives a better result.

In [ ]:
x, y = rng.multivariate_normal(mean, cov, 10000).T
print(np.mean(x))
print(np.mean(y))

$\unlhd$

Sampling from an arbitrary distribution on a finite set is also straightforward -- as long as the set is not too big. This can be done using [`numpy.random.Generator.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html). Borrowing the example from the documentation, the following:

In [ ]:
aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'christopher']
rng.choice(aa_milne_arr, 5, p=[0.5, 0.1, 0.1, 0.3])

generates $5$ samples from the set $\S = \{\tt{pooh}, \tt{rabbit}, \tt{piglet}, \tt{christopher}\}$ with respective probabilities $0.5, 0.1, 0.1, 0.3$.

But this may not be practical when the state space $\S$ is very large. As an example, later in this section, we will learn a "realistic" distribution of handwritten digits. We will do so using the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database), which we have encountered previously.

We load it from PyTorch and turn the grayscale images (more precisely, each pixel is an integer between $0$ and $255$) into a black-and-white images by rounding the pixels (after dividing by $255$).

In [ ]:
# Download and load the MNIST dataset
mnist = datasets.MNIST(root='./data', 
                       train=True, 
                       download=True, 
                       transform=transforms.ToTensor())

# Convert the dataset to a PyTorch DataLoader
train_loader = torch.utils.data.DataLoader(mnist, 
                                           batch_size=len(mnist), 
                                           shuffle=False)

# Extract images and labels from the DataLoader
imgs, labels = next(iter(train_loader))
imgs = imgs.squeeze().numpy()
labels = labels.numpy()

In [ ]:
imgs = np.round(imgs)

The first image is the following.

In [ ]:
plt.imshow(imgs[0], cmap=plt.cm.gray_r)
plt.show()

It is $28 \times 28$, so the total number of pixels is $784$.

In [ ]:
nx_pixels, ny_pixels = imgs[0].shape
nx_pixels, ny_pixels

In [ ]:
n_pixels = nx_pixels * ny_pixels
n_pixels

To specify the a distribution over all possible black and white images of this size, we need in principle to assign a probability to a very large number of states. Our space here is $\S = \{0,1\}^{784}$, imagining that $0$ encodes white and $1$ encodes black and that we have ordered the pixels in some arbitrary way. How big is this space?

Answer: $2^{784}$.

Or in base $10$, we compute $\log_{10}(2^{784})$, which is:

In [ ]:
784 * np.log(2) / np.log(10)

So a little more than $10^{236}$. 

This is much too large to naively plug into `rng.choice`!

**EXAMPLE:** Suppose $\S = \{1,\cdots, n\} = [n]$ for some positive integer $n$ and $\bpi$ is proportional to a Poisson distribution with mean $\lambda > 0$. That is, 

$$
\pi_i = C e^{-\lambda} \frac{\lambda^i}{i!}, \quad \forall i \in \S
$$

for some constant $C$ chosen so that $\sum_{i=1}^{n} \pi_i = 1$. Recall that we do not need to determine $C$ as it is enough to know the target distribution up to a scaling factor by the previous remark. 

To apply Metropolis-Hastings, we need a proposal chain. Consider the following choice. For each $1 < i < n$, move to $i+1$ or $i-1$ with probability $1/2$ each. For $i=1$ (respectively $i = n$), move to $2$ (respectively $n-1$) with probability $1/2$, otherwise stay where you are. For instance, if $n = 4$, then 

$$
Q
=
\begin{pmatrix}
1/2 & 1/2 & 0 & 0\\
1/2 & 0 & 1/2 & 0\\
0 & 1/2 & 0 & 1/2\\
0 & 0 & 1/2 & 1/2
\end{pmatrix},
$$

which is indeed a stochastic matrix. It is also symmetric, so it does not enter into the acceptance probability by the previous remark.

To compute the acceptance probability, we only need to consider pairs of adjacent integers as they are the only one that have non-zero probability under $Q$. Consider state $1 < i < n$. Observe that

$$
\frac{\pi_{i+1}}{\pi_{i}}
= \frac{C e^{-\lambda} \lambda^{i+1}/(i+1)!}{C e^{-\lambda} \lambda^{i}/i!}
= \frac{\lambda}{i+1}
$$

so a move to $i+1$ happens with probability

$$
\frac{1}{2} \min\left\{1, \frac{\lambda}{i+1}\right\},
$$

where the $1/2$ factor from the proposal distribution.
Similarly, it can be checked (try it!) that a move to $i-1$ occurs with probability

$$
\frac{1}{2} \min\left\{1, \frac{i}{\lambda}\right\}.
$$

And we stay at $i$ with probability $1 - \frac{1}{2} \min\left\{1, \frac{\lambda}{i+1}\right\} - \frac{1}{2} \min\left\{1, \frac{i}{\lambda}\right\}$. (Why is this guaranteed to be a probability?) 

A similar formula applies to $i = 1, n$. (Try it!)

We are ready to apply Metropolis-Hastings.  

In [ ]:
def mh_transition_poisson(lmbd, n):
    P = np.zeros((n,n))
    for idx in range(n):
        i = idx + 1 # index starts at 0 rather than 1
        if (i > 1 and i < n):
            P[idx, idx+1] = (1/2) * np.min(np.array([1, lmbd/(i+1)]))
            P[idx, idx-1] = (1/2) * np.min(np.array([1, i/lmbd]))
            P[idx, idx] = 1 - P[idx, idx+1] - P[idx, idx-1]
        elif i == 1:
            P[idx, idx+1] = (1/2) * np.min(np.array([1, lmbd/(i+1)]))
            P[idx, idx] = 1 - P[idx, idx+1]
        elif i == n:
            P[idx, idx-1] = (1/2) * np.min(np.array([1, i/lmbd]))
            P[idx, idx] = 1 - P[idx, idx-1]
    return P

Take $\lambda = 1$ and $n = 6$. We get the following transition matrix.

In [ ]:
lmbd = 1
n = 6

In [ ]:
P = mh_transition_poisson(lmbd, n)
print(P)

We use our simulator from a previous chapter. We start from the uniform distribution and take $100$ steps.

In [ ]:
mu = np.ones(n) / n
T = 100
X = mmids.SamplePath(mu, P, T)

Our sample is the final state of the trajectory.

In [ ]:
X[T]

We repeat $1000$ times and plot the frequencies.

In [ ]:
N_samples = 1000 # number of repetitions

freq_z = np.zeros(n) # init of frequencies sampled
for i in range(N_samples):
    X = mmids.SamplePath(mu, P, T)
    freq_z[int(X[T])-1] += 1 # adjust for index starting at 0
    
freq_z = freq_z/N_samples

In [ ]:
plt.bar(range(1,n+1),freq_z)
plt.show()

If we increase the parameter $\lambda$ (which is not quite the mean; why?), we expect the sampled distribution to shift to the right. We must recompute the transition matrix first.

In [ ]:
lmbd = 10
P = mh_transition_poisson(lmbd, n)

In [ ]:
freq_z = np.zeros(n) # init of frequencies sampled
for i in range(N_samples):
    X = mmids.SamplePath(mu, P, T)
    freq_z[int(X[T])-1] += 1 # adjust for index starting at 0
    
freq_z = freq_z/N_samples

In [ ]:
plt.bar(range(1,n+1),freq_z)
plt.show()

**TRY IT!** Redo the simulations, but this time implement a general Metropolis-Hastings algorithm rather than specifying the transition matrix directly. That is, implement the algorithm for an arbitrary $\bpi$ and $Q$. Assume the state space is $[n]$. ([Open in Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_prob_notebook.ipynb))

$\lhd$

### Gibbs sampling

**An example: restricted Boltzmann machines (RBM)** We implement the Gibbs sampler on a specific probabilistic model, a so-called restricted Boltzmann machine (RBM), and apply it to the generation of random images from a "realistic" distribution. For more on Boltzmann machines, including their restricted and deep versions, see [here](https://en.wikipedia.org/wiki/Boltzmann_machine). We will not describe them in great details here, but only use them as an example of a complex distribution.

*Probabilistic model:* An RBM has $m$ visible units (i.e., observed variables) and $n$ hidden units (i.e., hidden variables). It is represented by a complete bipartite graph between the two. 

Visible unit $i$ is associated a variable $v_i$ and hidden unit $j$ is associated a variable $h_j$. We define the corresponding vectors $\bv = (v_1,\ldots,v_m)$ and $\bh = (h_1,\ldots,h_n)$. For our purposes, it will suffice to assume that $\bv \in \{0,1\}^m$ and $\bh \in \{0,1\}^n$. These are referred to as binary units.

The probabilistic model has a number of parameters. Each visible unit $i$ has an offset $b_i \in \mathbb{R}$ and each hidden unit $j$ has an offset $c_j \in \mathbb{R}$. We write $\bb = (b_1,\ldots,b_m)$ and $\bc = (c_1,\ldots,c_n)$ for the offset vectors. For each pair $(i,j)$ of visible and hidden units (or, put differently, for each edge in the complete bipartite graph), there is a weight $w_{i,j} \in \mathbb{R}$. We write $W = (w_{i,j})_{i,j=1}^{m,n}$ for the weight matrix.

To define the probability distribution, we need the so-called [energy](https://en.wikipedia.org/wiki/Energy-based_model) (as you may have guessed, this terminology comes from related models in [physics](https://en.wikipedia.org/wiki/Boltzmann_distribution)): for $\bv \in \{0,1\}^m$ and $\bh \in \{0,1\}^n$,

\begin{align*}
\cE(\bv, \bh)
&= 
- \bv^T W \bh
- \bb^T \bv
- \bc^T \bh\\
&= 
- \sum_{i=1}^m \sum_{j=1}^n w_{i,j} v_i h_j
- \sum_{i=1}^m b_i v_i
- \sum_{j=1}^n c_j h_j.
\end{align*}

The joint distribution of $\bv$ and $\bh$ is

$$
\pi(\bv, \bh)
= \frac{1}{Z} \exp\left(- \cE(\bv, \bh)\right),
$$

where $Z$, the [partition function](https://en.wikipedia.org/wiki/Partition_function_%28statistical_mechanics%29) (a function of $W,\bb,\bc$), ensures that $\pi$ indeed sums to $1$.

We will be interested in sampling from the marginal over visible units, that is,

$$
\rho(\bv)
= \sum_{\bh \in \{0,1\}^n} \pi(\bv, \bh).
$$

When $m$ and/or $n$ are large, computing $\rho$ or $\pi$ explicitly -- or even numerically -- is impractical. 

We develop the Gibbs sampler for this model next.

*Gibbs sampling:* We sample from the joint distribution $\pi$ and observe only $\bv$.

We need to compute the conditional probabilities given every other variable. The sigmoid function, which we have encountered previously, 

$$
\sigma(x)
=
\frac{1}{1 + e^{-x}}
$$

will once again make an appearance.

In [ ]:
def sigmoid(x): 
    return 1/(1 + np.exp(-x))

In [ ]:
grid = np.linspace(-5, 5, 100)
plt.plot(grid,sigmoid(grid),'r')
plt.show()

Fix a visible unit $i \in [m]$. For a pair $(\bv, \bh)$, we denote by $(\bv_{[i]}, \bh)$ the same pair where coordinate $i$ of $\bv$ is flipped. Given every other variable, i.e., $(\bv_{-i},\bh)$, and using a superscript $\text{v}$ to indicate the probability of a visible unit, the conditional probability of $v_i$ is

\begin{align*}
\pi^{\text{v}}_i(v_i|\bv_{-i},\bh)
&= \frac{\pi(\bv, \bh)}{\pi(\bv, \bh) + \pi(\bv_{[i]}, \bh)}\\
&= \frac{\frac{1}{Z} \exp\left(- \cE(\bv, \bh)\right)}{\frac{1}{Z} \exp\left(- \cE(\bv, \bh)\right) + \frac{1}{Z} \exp\left(- \cE(\bv_{[i]}, \bh)\right)}.
\end{align*}

In this last ratio, the partition functions (the $Z$'s) cancel out. Moreover, all the terms in the exponentials *not depending* on the $i$-th visible unit actually factor out and cancel out as well -- they are identical in all three exponentials. Similarly, the terms in the exponentials *depending only on $\bh$* also factor out and cancel out.

What we are left with is:

\begin{align*}
&\pi^{\text{v}}_i(v_i|\bv_{-i},\bh)\\
&= \frac{\exp\left(\sum_{j=1}^n w_{i,j} v_i h_j
+ b_i v_i\right)}
{\exp\left(\sum_{j=1}^n w_{i,j} v_i h_j
+ b_i v_i\right) 
+ \exp\left(\sum_{j=1}^n w_{i,j} (1-v_i) h_j
+ b_i (1-v_i)\right)},
\end{align*}

where we used the fact that flipping $v_i \in \{0,1\}$ is the same as setting it to $1 - v_i$, a transformation which indeed sends $0$ to $1$ and $1$ to $0$.

This expression does not depend on $\bv_{-i}$. In other words, the $i$-th visible unit is conditionally independent of all other visible units given the hidden units. 

We simplify the expression

\begin{align*}
&\pi^{\text{v}}_i(v_i|\bv_{-i},\bh)\\
&= \frac{1}
{1 
+ \exp\left(\sum_{j=1}^n w_{i,j} (1-2 v_i) h_j
+ b_i (1- 2v_i)\right)}\\
&= \sigma\left(\sum_{j=1}^n w_{i,j} (2 v_i-1) h_j
+ b_i (2v_i-1)\right).
\end{align*}

In particular, the conditional mean of the $i$-th visible unit given everything else is 

\begin{align*}
0 \cdot \pi^{\text{v}}_i(0|\bv_{-i},\bh) + 1 \cdot \pi^{\text{v}}_i(1|\bv_{-i},\bh)
&= \pi^{\text{v}}_i(1|\bv_{-i},\bh)\\
&= \sigma\left(\sum_{j=1}^n w_{i,j} h_j
+ b_i \right)\\
&= \sigma\left((W \bh + \bb)_i
\right)
\end{align*}

Similarly for the conditional of the $j$-th hidden unit given everything else, we have

\begin{align*}
&\pi^{\text{h}}_j(h_j|\bv,\bh_{-j})\\
&= \sigma\left(\sum_{i=1}^m w_{i,j} v_i (2h_j -1) 
+ c_j (2h_j -1)\right).
\end{align*}

The conditional mean given everything else is 

\begin{align*}
0 \cdot \pi^{\text{h}}_j(0|\bv,\bh_{-j}) + 1 \cdot \pi^{\text{h}}_j(1|\bv,\bh_{-j})
&= \pi^{\text{h}}_j(1|\bv,\bh_{-j})
= \sigma\left((W^T \bv + \bc)_j \right).
\end{align*}

And the $j$-th hidden unit is conditionally independent of all other hidden units given the visible units. 

**NUMERICAL CORNER:** We implement the Gibbs sampler for an RBM. Rather than updating the units at random, we use a block approach. Specifically, we update all hidden units independently, given the visible units; then we update all visible units independently, given the hidden units. In each case, this is warranted by the conditional independence structure revealed above. 

We first implement the conditional means using the formulas previously derived. 

In [ ]:
def rbm_mean_hidden(v, W, c):
    return sigmoid(W @ v + c)

In [ ]:
def rbm_mean_visible(h, W, b):
    return sigmoid(W.T @ h + b)

We next implement one step of the sampler, which consists in updating all hidden units, followed by updating all visible units. 

In [ ]:
def rbm_gibbs_update(v, W, b, c):
    p_hidden = rbm_mean_hidden(v, W, c)
    h = rng.binomial(1, p_hidden, p_hidden.shape)
    p_visible = rbm_mean_visible(h, W, b)
    v = rng.binomial(1, p_visible, p_visible.shape)
    return v

Finally, we repeat these steps `k` times. We only return the visible units `v`.

In [ ]:
def rbm_gibbs_sampling(k, v_0, W, b, c):
    counter = 0
    v = v_0
    while counter < k:
        v = rbm_gibbs_update(v, W, b, c)
        counter += 1
    return v

Here `v_0` is the initial visible unit states. We do not need to initialize the hidden ones as this is done automatically in the first update step. In the next subsection, we will take the initial distribution of $\bv$ to be independent Bernoullis with success probability $1/2$.

$\unlhd$

### Sampling handwritten digits with an RBM

**Trainging the model** We first need to train the model on the data. We will not show how this is done here, but instead use [`sklearn.neural_network.BernoulliRBM`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html). (Some details of how this training is done is provided [here](https://scikit-learn.org/stable/modules/neural_networks_unsupervised.html#stochastic-maximum-likelihood-learning).)

In [ ]:
from sklearn.neural_network import BernoulliRBM

In [ ]:
rbm = BernoulliRBM(random_state=seed, verbose=0)

To simplify the analysis and speed up the training, we only keep digits $0$, $1$ and $5$.

In [ ]:
# Filter out images with labels 0, 1, or 5
mask = (labels == 0) | (labels == 1) | (labels == 5)
imgs = imgs[mask]
labels = labels[mask]

We flatten the images (which have already been "rounded" to black-and-white; see the first subsection).

In [ ]:
X = imgs.reshape(len(imgs), -1)

We now fit the model. Choosing the hyperparameters of the training algorithm is tricky. The following seem to work reasonably well. (For a more systematic approach to tuning hyperparameters, see [here](https://scikit-learn.org/stable/modules/grid_search.html).)

In [ ]:
rbm.n_components = 100
rbm.learning_rate = 0.02
rbm.batch_size = 50
rbm.n_iter = 20
rbm.fit(X)

We plot the learned weight matrix using a script [adapted from here](https://scikit-learn.org/stable/auto_examples/neural_networks/plot_rbm_logistic_classification.html). Each image shows the weights associated to all visible units by one hidden unit.

In [ ]:
def plot_imgs(z, n_imgs, nx_pixels, ny_pixels):
    nx_imgs = np.floor(np.sqrt(n_imgs))
    ny_imgs = np.ceil(np.sqrt(n_imgs))
    plt.figure(figsize=(8, 8))
    for i, comp in enumerate(z):
        plt.subplot(int(nx_imgs), int(ny_imgs), i + 1)
        plt.imshow(comp.reshape((nx_pixels, ny_pixels)), 
                   cmap=plt.cm.gray_r)
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
plot_imgs(rbm.components_, rbm.n_components, 
          nx_pixels, ny_pixels)

**Back to Gibbs sampling** We are ready to sample from the trained RBM. We extract the learned parameters from `rbm`.

In [ ]:
W = rbm.components_
W.shape

In [ ]:
b = rbm.intercept_visible_
b.shape

In [ ]:
c = rbm.intercept_hidden_
c.shape

To generate $25$ samples, we first generate $25$ independent initial states. We stack them into a matrix, where each row is a different flattened random noise image.

In [ ]:
n_samples = 25 # number of samples
z = rng.binomial(1, 0.5, (n_samples, n_pixels))

In [ ]:
plot_imgs(z, n_samples, nx_pixels, ny_pixels)

To process all samples simultaneously, we make a small change to the code. We [`numpy.reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)
to make the offsets into column vectors, which are then automatically added to all columns of the resulting weighted sum. 
(This is known as [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html).)

In [ ]:
def rbm_mean_hidden(v, W, c):
    return sigmoid(W @ v + c.reshape(len(c),1))

In [ ]:
def rbm_mean_visible(h, W, b):
    return sigmoid(W.T @ h + b.reshape(len(b),1))

We are now ready to run our Gibbs sampler. The outcome depends on the number of steps we take. For instance, after one step, the result is still very noisy -- although note that the fraction of white pixels is already more realistic!

In [ ]:
v_0 = z.T
gen_v = rbm_gibbs_sampling(1, v_0, W, b, c)

In [ ]:
plot_imgs(gen_v.T, n_samples, nx_pixels, ny_pixels)

After $10$ steps, we already see shadows of digits appearing. 

In [ ]:
v_0 = z.T
gen_v = rbm_gibbs_sampling(10, v_0, W, b, c)

In [ ]:
plot_imgs(gen_v.T, n_samples, nx_pixels, ny_pixels)

After $100$ steps, the outcome is quite realistic. 

In [ ]:
v_0 = z.T
gen_v = rbm_gibbs_sampling(100, v_0, W, b, c)

In [ ]:
plot_imgs(gen_v.T, n_samples, nx_pixels, ny_pixels)

RBMs can be stacked into [deep probabilistic models](https://en.wikipedia.org/wiki/Boltzmann_machine#Deep_Boltzmann_machine), similarly to what we encountered with neural networks.

$\newcommand{\P}{\mathbb{P}}$ $\newcommand{\E}{\mathbb{E}}$ $\newcommand{\S}{\mathcal{S}}$ $\newcommand{\var}{\mathrm{Var}}$ $\newcommand{\bmu}{\boldsymbol{\mu}}$ $\newcommand{\bSigma}{\boldsymbol{\Sigma}}$ $\newcommand{\btheta}{\boldsymbol{\theta}}$ $\newcommand{\bpi}{\boldsymbol{\pi}}$ $\newcommand{\indep}{\perp\!\!\!\perp}$ $\newcommand{\bp}{\mathbf{p}}$ $\newcommand{\bx}{\mathbf{x}}$ $\newcommand{\bX}{\mathbf{X}}$ $\newcommand{\by}{\mathbf{y}}$ $\newcommand{\bY}{\mathbf{Y}}$ $\newcommand{\bz}{\mathbf{z}}$ $\newcommand{\bZ}{\mathbf{Z}}$ $\newcommand{\bw}{\mathbf{w}}$ $\newcommand{\bW}{\mathbf{W}}$ $\newcommand{\bv}{\mathbf{v}}$ $\newcommand{\bV}{\mathbf{V}}$ $\newcommand{\bfbeta}{\boldsymbol{\beta}}$ $\newcommand{\bflambda}{\boldsymbol{\lambda}}$

## More advanced material

### Cholesky decomposition

**NUMERICAL CORNER:** We implement the algorithm above. In our naive implementation, we assume that $B$ is positive definite, and therefore that all steps are well-defined.

In [ ]:
def cholesky(B):
    n = B.shape[0] 
    L = np.zeros((n, n))
    for j in range(n):
        L[j,0:j] = mmids.forwardsubs(L[0:j,0:j],B[j,0:j])
        L[j,j] = np.sqrt(B[j,j] - LA.norm(L[j,0:j])**2)
    return L 

Here is a simple example.

In [ ]:
B = np.array([[2., 1.],[1., 2.]])
print(B)

In [ ]:
L = cholesky(B)
print(L)

We can check that it produces the right factorization.

In [ ]:
print(L @ L.T)

$\unlhd$

**NUMERICAL CORNER:** We implement this algorithm below. In our naive implementation, we assume that $A$ has full column rank, and therefore that all steps are well-defined.

In [ ]:
def ls_by_chol(A, b):
    L = cholesky(A.T @ A)
    z = mmids.forwardsubs(L, A.T @ b)
    return mmids.backsubs(L.T, z)

$\unlhd$